# imports

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig, PreTrainedModel, LlamaTokenizer, StoppingCriteria
from dotenv import load_dotenv
import os
from datasets import load_from_disk
from trl import SFTTrainer
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model, AutoPeftModelForCausalLM, PeftModel
import torch

In [4]:
#load hugging-face token
load_dotenv()
hf_token = os.environ["HF_ACCESS_TOKEN"]

#set wandb environment variables
os.environ["WANDB_ENTITY"] = "t_buess"
os.environ["WANDB_PROJECT"] = "chatbot-qa"

# parameters

In [5]:
base_model_id = "meta-llama/Llama-2-7b-hf"
finetuned_path = "./data/models/llama2-fine-tuned"
ft_dataset_filename = "data/processed/ft_dataset.hf"
train_set_text_field = "text"
train_batch_size = 1
grad_accumulation_steps = 8
optimizer = "paged_adamw_32bit"
learning_rate = 1e-4
max_steps = 200
max_seq_length = 4096
device_map="auto"

# configuration

In [4]:
lora_config = LoraConfig(
    r=16,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

# load tokenizer and model

In [5]:
tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    token=hf_token,
)
tokenizer.pad_token = "</p>"
tokenizer.padding_side = "right"

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    quantization_config=bnb_config,
    device_map=device_map,
    token=hf_token,
)
base_model.config.use_cache = False
base_model.config.pretraining_tp = 1

base_model:AutoPeftModelForCausalLM = prepare_model_for_kbit_training(base_model)
base_model = get_peft_model(base_model, lora_config) # add lora adapters
base_model.print_trainable_parameters()

Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.18s/it]
c:\Users\Stephan\anaconda3\envs\chatbot-qa\Lib\site-packages\transformers\utils\hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


trainable params: 8,388,608 || all params: 6,746,804,224 || trainable%: 0.12433454005023165


# load dataset

In [6]:
def formatter(example):
    prompt = (
        "Nachfolgend ist eine Frage gestellt mit dem entsprechenden Kontext sowie der passenden Antwort"
        "Schreibe eine passende Antwort zur Frage und beziehe den Kontext mit hinein"
        "### Frage:\n"
        f"{example['question']}\n\n"
        "### Kontext:\n"
        f"{example['context']}\n\n"
        "### Antwort:\n"
        f"{example['answers']}{tokenizer.eos_token}"
    )

    return {"text": prompt}

#load train dataset
train_dataset = load_from_disk(ft_dataset_filename)

#add text column
train_dataset = train_dataset.map(formatter)

# train

In [ ]:
train_args = TrainingArguments(
    output_dir=finetuned_path + "/train-out",
    per_device_train_batch_size=train_batch_size,
    gradient_accumulation_steps=grad_accumulation_steps,
    learning_rate=learning_rate,
    optim=optimizer,
    logging_steps=1,
    max_steps=max_steps,
    lr_scheduler_type="constant",
    group_by_length=True
)

fine_tuning = SFTTrainer(
    model=base_model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    dataset_text_field=train_set_text_field,
    max_seq_length=max_seq_length,
    args=train_args
)

fine_tuning.train()

# save model and tokenizer

In [8]:
fine_tuning.model.save_pretrained(finetuned_path + "/model")
fine_tuning.tokenizer.save_pretrained(finetuned_path + "/tokenizer")

('./data/models/llama2-fine-tuned/tokenizer\\tokenizer_config.json',
 './data/models/llama2-fine-tuned/tokenizer\\special_tokens_map.json',
 './data/models/llama2-fine-tuned/tokenizer\\tokenizer.model',
 './data/models/llama2-fine-tuned/tokenizer\\added_tokens.json',
 './data/models/llama2-fine-tuned/tokenizer\\tokenizer.json')

# inference

In [7]:
def predict(model:PreTrainedModel, tokenizer:LlamaTokenizer, question:str, context:str):
    model.eval()
    
    prompt = (
        "Nachfolgend ist eine Frage gestellt mit dem entsprechenden Kontext\n"
        "Schreibe eine passende Antwort als vollständiger Satz zur Frage und beziehe den Kontext mit hinein\n\n"
        "### Frage:\n"
        f"{question}\n\n"
        "### Kontext:\n"
        f"{context}\n\n"
        "### Antwort:\n"
    )

    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(input_ids=inputs["input_ids"].to("cuda:0"), attention_mask=inputs["attention_mask"], max_new_tokens=200, pad_token_id=tokenizer.pad_token_id, do_sample=True)

    return tokenizer.decode(outputs[:, inputs["input_ids"].shape[1]:][0], skip_special_tokens=True)

In [39]:
question = "Was was die erste Version von Python?"
context = (
    "Die Sprache wurde Anfang der 1990er Jahre von Guido van Rossum am Centrum Wiskunde & Informatica in Amsterdam als Nachfolger für die Programmier-Lehrsprache ABC entwickelt" 
    "und war ursprünglich für das verteilte Betriebssystem Amoeba gedacht."
    "Der Name geht nicht, wie das Logo vermuten lässt, auf die gleichnamige Schlangengattung Python zurück, sondern bezog sich ursprünglich auf die englische Komikergruppe Monty Python." 
    "In der Dokumentation finden sich daher auch einige Anspielungen auf Sketche aus dem Flying Circus. Trotzdem etablierte sich die Assoziation zur Schlange, was sich unter anderem" 
    "in der Programmiersprache Cobra[16] sowie dem Python-Toolkit „Boa“[17] äußert. Die erste Vollversion erschien im Januar 1994 unter der Bezeichnung Python 1.0." 
    "Gegenüber früheren Versionen wurden einige Konzepte der funktionalen Programmierung implementiert, die allerdings später wieder aufgegeben wurden.[18] Von 1995 bis 2000 erschienen neue Versionen,"
    "die fortlaufend als Python 1.1, 1.2 etc. bezeichnet wurden."
)

print("Frage: ", question)
print("Antwort: ", predict(base_model, tokenizer, question, context))

'Python 1.0'

# load model

In [9]:
#load hugging-face token
load_dotenv()
hf_token = os.environ["HF_ACCESS_TOKEN"]

#load lora config from model
lora_config = LoraConfig.from_pretrained(finetuned_path + "/model") 

#define bnb config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

ft_model = AutoModelForCausalLM.from_pretrained(
    lora_config.base_model_name_or_path, 
    quantization_config=bnb_config, 
    device_map="cuda:0"
)
ft_model = PeftModel.from_pretrained(ft_model, finetuned_path + "/model")

ft_tokenizer = AutoTokenizer.from_pretrained(finetuned_path + "/tokenizer")

question = "Was was die erste Version von Python?"
context = (
    "Die Sprache wurde Anfang der 1990er Jahre von Guido van Rossum am Centrum Wiskunde & Informatica in Amsterdam als Nachfolger für die Programmier-Lehrsprache ABC entwickelt" 
    "und war ursprünglich für das verteilte Betriebssystem Amoeba gedacht."
    "Der Name geht nicht, wie das Logo vermuten lässt, auf die gleichnamige Schlangengattung Python zurück, sondern bezog sich ursprünglich auf die englische Komikergruppe Monty Python." 
    "In der Dokumentation finden sich daher auch einige Anspielungen auf Sketche aus dem Flying Circus. Trotzdem etablierte sich die Assoziation zur Schlange, was sich unter anderem" 
    "in der Programmiersprache Cobra[16] sowie dem Python-Toolkit „Boa“[17] äußert. Die erste Vollversion erschien im Januar 1994 unter der Bezeichnung Python 1.0." 
    "Gegenüber früheren Versionen wurden einige Konzepte der funktionalen Programmierung implementiert, die allerdings später wieder aufgegeben wurden.[18] Von 1995 bis 2000 erschienen neue Versionen,"
    "die fortlaufend als Python 1.1, 1.2 etc. bezeichnet wurden."
)

print("Frage: ", question)
print("Antwort: ", predict(ft_model, ft_tokenizer, question, context))

Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.01s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Frage:  Was was die erste Version von Python?
Antwort:  Python 1.0
